In [1]:
# Get the data here: https://data.world/dataquest/mlb-game-logs
import sys, os
sys.path.insert(0, os.path.abspath("../.."))
%load_ext ElasticNotebook

/mnt/c/Users/RECODE/Programs/ElasticNotebook/.linux-venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:548: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [2]:
%%RecordEvent
import pandas as pd
gl = pd.read_csv('data/game_logs.csv')
gl.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,20230330,0,Thu,MIL,NL,1,CHN,NL,1,0,...,Eric Hosmer,3,wisdp001,Patrick Wisdom,5,mastm001,Miles Mastrobuoni,9,NaN,Y
1,20230330,0,Thu,PIT,NL,1,CIN,NL,1,5,...,Spencer Steer,5,bensw001,Will Benson,7,garcj007,Jose Garcia,6,NaN,Y
2,20230330,0,Thu,ARI,NL,1,LAN,NL,1,2,...,Miguel Vargas,4,outmj002,James Outman,8,rojam002,Miguel Rojas,6,NaN,Y
3,20230330,0,Thu,NYN,NL,1,MIA,NL,1,5,...,Bryan De La Cruz,7,stalj001,Jacob Stallings,2,wendj002,Joey Wendle,6,NaN,Y
4,20230330,0,Thu,COL,NL,1,SDN,NL,1,7,...,Austin Nola,2,kim-h002,Ha-Seong Kim,4,dahld001,David Dahl,9,NaN,Y


In [3]:
%%RecordEvent
gl = gl.fillna(0)
gl.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2430 entries, 0 to 2429
Columns: 161 entries, date to acquisition_info
dtypes: float64(5), int64(82), object(74)
memory usage: 12.8 MB


In [4]:
%%RecordEvent
gl.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,20230330,0,Thu,MIL,NL,1,CHN,NL,1,0,...,Eric Hosmer,3,wisdp001,Patrick Wisdom,5,mastm001,Miles Mastrobuoni,9,0,Y
1,20230330,0,Thu,PIT,NL,1,CIN,NL,1,5,...,Spencer Steer,5,bensw001,Will Benson,7,garcj007,Jose Garcia,6,0,Y
2,20230330,0,Thu,ARI,NL,1,LAN,NL,1,2,...,Miguel Vargas,4,outmj002,James Outman,8,rojam002,Miguel Rojas,6,0,Y
3,20230330,0,Thu,NYN,NL,1,MIA,NL,1,5,...,Bryan De La Cruz,7,stalj001,Jacob Stallings,2,wendj002,Joey Wendle,6,0,Y
4,20230330,0,Thu,COL,NL,1,SDN,NL,1,7,...,Austin Nola,2,kim-h002,Ha-Seong Kim,4,dahld001,David Dahl,9,0,Y


In [5]:
%%RecordEvent
for dtype in ['float','int','object']:
    selected_dtype = gl.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

Average memory usage for float columns: 0.02 MB
Average memory usage for int columns: 0.02 MB
Average memory usage for object columns: 0.15 MB


In [6]:
%%RecordEvent
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [7]:
%%RecordEvent
gl_int = gl.select_dtypes(include=['int'])
converted_int = gl_int.apply(pd.to_numeric,downcast='unsigned')
print(mem_usage(gl_int))
print(mem_usage(converted_int))
compare_ints = pd.concat([gl_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

1.52 MB
0.20 MB


,before,after
uint8,NaN,80.0
uint16,NaN,1.0
uint32,NaN,1.0
int64,82.0,NaN


In [8]:
%%RecordEvent
gl_float = gl.select_dtypes(include=['float'])
converted_float = gl_float.apply(pd.to_numeric,downcast='float')
print(mem_usage(gl_float))
print(mem_usage(converted_float))
compare_floats = pd.concat([gl_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

0.09 MB
0.05 MB


,before,after
float32,NaN,5.0
float64,5.0,NaN


In [9]:
%%RecordEvent
optimized_gl = gl.copy()
optimized_gl[converted_int.columns] = converted_int
optimized_gl[converted_float.columns] = converted_float

print(mem_usage(gl))
print(mem_usage(optimized_gl))

12.77 MB
11.40 MB


In [10]:
%%RecordEvent
gl_obj = gl.select_dtypes(include=['object']).copy()
gl_obj.describe()

,day_of_week,v_name,v_league,h_name,h_league,day_night,completion,park_id,v_line_score,h_line_score,...,h_player_6_id,h_player_6_name,h_player_7_id,h_player_7_name,h_player_8_id,h_player_8_name,h_player_9_id,h_player_9_name,additional_info,acquisition_info
count,2430,2430,2430,2430,2430,2430,2430,2430,2430,2430,...,2430,2430,2430,2430,2430,2430,2430,2430,2430,2430
unique,7,30,2,30,2,2,7,33,1514,1624,...,380,379,418,418,395,394,331,331,13,1
top,Sat,MIL,NL,CHN,NL,N,0,CHI11,000000000,000000000,...,heimj001,Jonah Heim,brenw002,Will Brennan,arcio002,Orlando Arcia,maldm001,Martin Maldonado,0,Y
freq,412,81,1215,81,1215,1501,2424,81,169,132,...,48,48,30,30,55,55,58,58,2417,2430


In [11]:
%%RecordEvent
dow = gl_obj.day_of_week
print(dow.head())
dow_cat = dow.astype('category')
print(dow_cat.head())

0    Thu
1    Thu
2    Thu
3    Thu
4    Thu
Name: day_of_week, dtype: object
0    Thu
1    Thu
2    Thu
3    Thu
4    Thu
Name: day_of_week, dtype: category
Categories (7, object): ['Fri', 'Mon', 'Sat', 'Sun', 'Thu', 'Tue', 'Wed']


In [12]:
%%RecordEvent
converted_obj = pd.DataFrame()
for col in gl_obj.columns:
    num_unique_values = len(gl_obj[col].unique())
    num_total_values = len(gl_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = gl_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = gl_obj[col]

In [13]:
%Checkpoint checkpoints/pandas.pickle

migration speed (bps): 690692167.7980205


ValueError: buffer source array is read-only

In [ ]:
%who